# Network Intrusion Detection with Deep Learning

In [0]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3" #model will be trained on GPU 0

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## The Data

# New Section

In [0]:
import numpy as np
def load_matrices(file_name):
    with open(file_name, 'rb') as f:
        A = np.load(f)
        B = np.load(f)
        C = np.load(f)
        D = np.load(f)
        E = np.load(f)
        F = np.load(f)
    return (A,B,C,D,E,F)

In [0]:
my_file =  '10data2.npy'
train,test,y_train,y_test,ylabel_train, ylabel_test = load_matrices(my_file)

## The Model

In [10]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Sequential
#from keras.layers import Dense, Activation, Merge, Reshape, Dropout
from keras.layers import Dense, Activation, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import InputLayer, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.models import load_model

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live

from keras.layers import Dropout, Dense

# Keras items
from keras.activations import relu, elu
from keras.losses import binary_crossentropy


In [0]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU
from keras.activations import elu


In [0]:
def selu(x):
    """Scaled Exponential Linear Unit. (Klambauer et al., 2017)
    # Arguments
        x: A tensor or variable to compute the activation function for.
    # References
        - [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)
    """
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale * elu(x, alpha)


In [0]:
#actvation leakyrelu
LR= LeakyReLU(0.01)
LR.__name__ = 'lrelu'
#actvation prelu
#PR= PReLU(init='zero', weights=None)
#PR.__name__ = 'prelu'

In [0]:
from keras.callbacks import ModelCheckpoint, CSVLogger

In [0]:
def log_file_name1(first_neuron,batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'

    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log1layerb/neuron1_{0}-batch{1}-kernelinit_{2}-lr{3}-activ_{4}.hdf"
    log_file = s.format(first_neuron,batch_size,kernel_initial,lr,activation)
    return log_file

def log_file_name2(first_neuron,batch_size,kernel_initial,lr,activation):
    
    if activation==LR:
       activation= 'lrelu'
    
    if activation==PReLU:
       activation= 'prelu'
    
    if activation==relu:
       activation= 'relu'
    
    if activation==elu:
       activation= 'elu'
    
    if activation==selu:
       activation= 'selu'
    
    # The dir-name for the TensorBoard log-dir.
    s = "./logtalos/log1layerb/neuron1_{0}-batch{1}-kernelinit_{2}-lr{3}-activ_{4}.csv"
    log_file = s.format(first_neuron,batch_size,kernel_initial,lr,activation)
    return log_file


In [0]:
from keras.callbacks import ModelCheckpoint
filepath="./logtalos/1layertes_b.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
from keras.models import Model, Sequential

In [0]:
# first we have to make sure to input data and params into the function
# add input parameters to the function
def dnn(x_train, ylabel_train, x_val, y_val, params):

    input_dim = x_train.shape[1]

    input_img = Input(shape=(input_dim,))
    
    first_neuron=params['first_neuron']
    batch_size=params['batch_size']
    kernel_initial=params['kernel_initializer']
    lr=params['lr']
    activation=params['activation']
    
    print('activation=',activation)
    
    log_file1 = log_file_name1(first_neuron,batch_size,kernel_initial,lr,activation)
    checkpoint2 = ModelCheckpoint(log_file1, monitor='acc', save_best_only=True, mode='max')

    
    log_file2 = log_file_name2(first_neuron,batch_size,kernel_initial,lr,activation)

    csv_logger2 = CSVLogger(log_file2,separator=',', append=False)
    callbacks_list2 = [checkpoint2,csv_logger2]
    
    ###AE
    
    # replace the hyperparameter inputs with references to params dictionary 
    autoencoder = Sequential()
    #encoder
    
    if activation==PReLU: 
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), kernel_initializer=kernel_initial))
        autoencoder.add(PReLU())
        
                   
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
        
    else:
        autoencoder.add(Dense(first_neuron, input_shape=(input_dim,), activation=activation, kernel_initializer=kernel_initial))
        #decoder
        #model.add(Dense(8, activation=params['activation']))
        autoencoder.add(Dense(input_dim, activation='sigmoid', kernel_initializer=kernel_initial))
    
    #model = multi_gpu_model(model, gpus=1)
    autoencoder.summary()
    print(input_dim)
    
    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
   
    encoder = Model(input_img, encoder_layer1(input_img))
    
    #encoder_layer1 = autoencoder.layers[0]
    #encoder_layer2 = autoencoder.layers[1]
    #encoder_layer3 = autoencoder.layers[2]
    #encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    encoder.summary()

    
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['acc'])
    #model.compile(loss=params['losses'],
    #              optimizer=params['optimizer'](),
    #              metrics=['acc', fmeasure_acc])
    
   

    
    history=autoencoder.fit(x,x,
                             epochs=params['epochs'],
                             batch_size=batch_size,
                              #shuffle=True,
                              validation_split=0.2,
                              callbacks=callbacks_list,                    
                             #callbacks=[early_stopping])
                              #validation_data=(test,test),
                              verbose=0)
  

    num_classes=15
    out2 = Dense(num_classes, activation='softmax',kernel_initializer=kernel_initial)(encoder.output)
    newmodel = Model(encoder.input,out2)
    
   
    newmodel.compile(loss='categorical_crossentropy', 
                     #optimizer='adam', 
                     optimizer= Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08,decay=0.0, amsgrad=False),
                     metrics=['acc']) 
    newmodel.summary()


    history2a=newmodel.fit(x, y,
                           epochs=params['epochs2'],
                           batch_size=batch_size,
                           validation_split=0.2,
                           callbacks=callbacks_list2,
                           #callbacks=[early_stopping],
                           #validation_data=(test,y_test),
                           verbose=0)

    
    del encoder
    del autoencoder
    #del dnn_model
    
    tf.reset_default_graph()
    
    return history2a, newmodel #, encoder
    

In [0]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[60,50, 40,30],
     'batch_size': [32,64,256],
     'epochs': [20],
     'epochs2': [30],
     'dropout': [0],
     'kernel_initializer': ['lecun_uniform','lecun_normal','he_normal','he_uniform','normal','glorot_uniform','glorot_normal'],
     'optimizer': [Adam],
     'lr':[0.1,0.01, 0.001,0.0001,0.00001],
     'activation':[relu, elu,selu,  LR, PReLU]}

In [0]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session

config = tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [0]:
x=train
y=ylabel_train

In [28]:
np.shape(x)

(1619315, 80)

In [29]:
np.shape(y)

(1619315, 15)

In [0]:

t = ta.Scan(x, y, p, dnn, experiment_no='11',grid_downsample=0.1,)


  0%|          | 0/33 [00:00<?, ?it/s]

activation= <function relu at 0x7f667712dd08>
Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                4080      
Total params: 8,130
Trainable params: 8,130
Non-trainable params: 0
_________________________________________________________________
80
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                4050      
Total params: 4,050
Trainable params: 4,050
No

  3%|▎         | 1/33 [23:47<12:41:30, 1427.82s/it]

activation= <function relu at 0x7f667712dd08>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                3280      
Total params: 6,520
Trainable params: 6,520
Non-trainable params: 0
_________________________________________________________________
80
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                3240      
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
__________________________________________________

  6%|▌         | 2/33 [58:35<14:00:00, 1625.83s/it]

...

activation= <function relu at 0x7f667712dd08>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                4080      
Total params: 8,130
Trainable params: 8,130
Non-trainable params: 0
_________________________________________________________________
80
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                4050      
Total params: 4,050
Trainable params: 4,050
Non-trainable params: 0
__________________________________________________

 73%|███████▎  | 24/33 [9:05:18<4:05:55, 1639.53s/it]

activation= <function relu at 0x7f667712dd08>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                4860      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                4880      
Total params: 9,740
Trainable params: 9,740
Non-trainable params: 0
_________________________________________________________________
80
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)                4860      
Total params: 4,860
Trainable params: 4,860
Non-trainable params: 0
__________________________________________________

 76%|███████▌  | 25/33 [9:30:36<3:33:43, 1602.96s/it]

activation= <function relu at 0x7f667712dd08>
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                3280      
Total params: 6,520
Trainable params: 6,520
Non-trainable params: 0
_________________________________________________________________
80
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                3240      
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
__________________________________________________

In [0]:
t.data

In [0]:
r = ta.Reporting(t)

In [0]:
r.best_params()